In [2]:
# Download and import the necessary libraries

!pip install torch
!pip install numpy
!pip install spektral # for loading and preprocessing the Cora dataset in a nice format

import torch as t
import numpy as np
import spektral


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.0 MB/s eta 0:00:00


# This notebook demos several techniques to implement Graph neural networks (GNN) and eventually uses those techniques to perform classification on the Cora dataset.

# Cora

Cora is a citation network. Each node in the network is a paper, and the edges between papers are citations. The purpose of the Cora benchmark is to classify papers into topics. Cora dataset details:

- 2708 nodes
- 5429 edges
- Features are a bag-of-words
- 7 topics
- 140 training papers
- 500 validation papers
- 1000 test papers


# Why?

Graphs are everywhere. Graphs can represent chemical bonds, social media connections, transportation maps, and so much more. Much like regular neural networks, graph neural networks allow us to perform classifcation tasks and help us make informative decisions on structures that take the form of a graph.

# Why not just use regular neural nets?

Neural nets work really well with data that has a defined structure and order. For example, consider a classifier that works on the MNIST dataset. Each input in this dataset is an image, and images have a well defined structure: a NxN grid of pixels each with well defined values. The order of the surrounding pixels is also well defined. Graphs on the other hand, do not have a specific shape or predefined order. Thus it makes sense to use a different kind of neural network to learn about the connections in the graph.

# How are GNNs implemented?

The idea for GNNs are actually very similar to convolutional neural networks that work on images. In fact, we can think of an image as a graph of pixels where each pixel is connected to their 8 adjacent neighbors. When we perform a convolution, we take a kernel over these neighbors and the pixel in question, perform an element wise multiplication and aggregate these values to replace the value of the pixel in question. This idea is very similar to how we will approach a GNN.

# How to implement the update rule?

There are several techniques to implement the update rule for a GNN. The general form is:

$$\vec{h'_{i}} = \sigma * \sum_{j\in N_i}a_{ij}W\vec{h_j}$$

Where $a_{ij}$ is a coefficient for each node in the graph. In other words, this is how important node $j$'s features are to node $i$. There are several common forms for that $a_{ij}$ can take

# Sum-Pooling and Mean Sum-Pooling

With basic sum-pooling, the coefficient $a_{ij}$ will take the value of 1. This will essentially just sum up the features of node $i$'s neighbors during update. However, this can cause the scale of the output features to increase. Thus, we need to normalize - introducing the mean sum-pooling form. With mean sum-pooling, the coefficient $a_{ij}$ takes the form $\frac{1}{N_i}$. This provides a more robust approach and is very useful for induction problems.

# GCN

If we want to use symmetric normalization instead, the coefficient $a_{ij}$ will take the form: $\frac{1}{\sqrt{|N_i||N_j|}}$. This form currently represents the most popular graph convolutional layer. Benefits: Easy to scale and really powerful. Cons: Not general enough. Cannot handle complex features.

# How to make the update rule more general

Focus on edge-wise mechanisms. In other words we can send messages along the edges, which can be conditioned by edge features and then aggregate these messaages to update our node. This method is called a message-passing neural network (MPNN). The update rule for MPNNs take the form:

$$\vec{h'_{i}} = f_v(\vec{h_i}\sum_{j\in N_i}\vec{m_{ji}})$$

where $f_v$ is some readout function, and $m_{ji}$ is a message from node $j$ to node $i$. We compute the message using a function $f_e$. $f_v$ and $f_e$ are usually small MLPs.  

An MPNN is the most potent GNN layer (of the ones that only look at first order neighbors) but it has a few cons:

- Requires storage and manipulation of messages
- Can be memory-wise and representionally troublesome. Has the potential to really overfit the data or take up a lot of memory.

Thus, they are only really applicable for small graphs.

# GAT

With GATs, we can calculate the coefficient $a_{ij}$ implicitly using some attention function $a$. $a$ can be any learnable, shared, self-attention mechanism. They may not be as general as MPNNs but they can scaled up by a lot.

# Note: Transformers are GNNs! More specifically they are MPNNs.



# Cora

Cora is a citation network. Each node in the network is a paper, and the edges between papers are citations. The purpose of the Cora benchmark is to classify papers into topics. Cora dataset details:

- 2708 nodes
- 5429 edges
- Features are a bag-of-words
- 7 topics
- 140 training papers
- 500 validation papers
- 1000 test papers

Let's load the Cora dataset and look at some of its properties.

In [27]:
# Load the Cora dataset
cora = spektral.datasets.citation.Citation('cora')

# Print the number of nodes and labels
print("Number of Nodes:", cora.n_nodes)
print("Number of Node Features:", cora.n_node_features)
print("Number of Labels:", cora.n_labels)





Number of Nodes: 2708
Number of Node Features: 1433
Number of Labels: 7


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
